# Demo with `tiled.client` (Python) API

Demonstrate, using the `tiled.client` Python API, how to select runs between a set of dates and then by plan name.

In [1]:
import datetime
from tiled.client import from_uri
from tiled.client.cache import Cache
from tiled.utils import tree
import tiled.queries

In [2]:
tiled_server = "localhost"
tiled_server_port = 8000

In [3]:
# connect our client with the server
uri = f"http://{tiled_server}:{tiled_server_port}"
print(f"{uri=}")
client = from_uri(uri, cache=Cache.in_memory(2e9))
print(f"{client=}")

uri='http://localhost:8000'
client=<Node {'training', 'class_2021_03', 'files'}>


In [4]:
# use the class catalog
cat = client["class_2021_03"]
print(f"{cat=}")

cat=<Node {'83cae163-e32b-4e62-887b-66f03f61ec33', ...} ~40 entries>


In [5]:
# define some helper functions which make simplify the searches
def iso2time(isotime):
    return datetime.datetime.timestamp(datetime.datetime.fromisoformat(isotime))

def QueryTimeSince(isotime):
    return tiled.queries.Key("time") >= iso2time(isotime)

def QueryTimeUntil(isotime):
    return tiled.queries.Key("time") < iso2time(isotime)

In [6]:
# Find all runs in the catalog between these two ISO8601 dates.
# (Show ALL the steps to convert the dates to timestamps for the tiled query.)
# Later, we'll build a small utility module to make these searches look easier.
start_time = "2021-03-17 00:30"
end_time = "2021-05-19 15:15"
cat = cat.search(  # since
    tiled.queries.Key("time") >= datetime.datetime.timestamp(
        datetime.datetime.fromisoformat(start_time)
    )
).search(  # until
    tiled.queries.Key("time") < datetime.datetime.timestamp(
        datetime.datetime.fromisoformat(end_time)
    )
)
print(f"{cat=}")

cat=<Node {'d6e1bad1-92f6-4257-8d5e-53172cd9b784', ...} ~12 entries>


In [7]:
# Filter this `cat` further.  Find run(s) which match given metadata: given plan_name.
plan_name = "rel_scan"
cat = cat.search(tiled.queries.Key("plan_name") == plan_name)
print(f"{cat=}")

cat=<Node {'d6e1bad1-92f6-4257-8d5e-53172cd9b784', ...} ~6 entries>


In [8]:
# With last (latest) run:
run = cat.values()[-1]
print(f"last run: {run=}")

last run: run=<Node {'baseline', 'primary'}>


In [9]:
# Get overall metadata from this run.
print(f"Run metadata: {run.metadata=}")

Run metadata: run.metadata=DictView({'start': {'uid': 'a4c77cc5-dd49-4fc2-90e6-e3cd92667037', 'time': 1621455296.577983, 'login_id': 'prjemian@zap', 'scan_id': 188, 'pid': 3712584, 'objective': 'Demonstrate UB matrix save & restore', 'notebook': 'UB_autosave', 'beamline_id': 'APS_Python_training_2021', 'versions': {'apstools': '1.5.0rc1', 'bluesky': '1.6.7', 'databroker': '1.2.2', 'epics': '3.4.3', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.1', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}, 'instrument_name': 'class_2021_03', 'proposal_id': 'training', 'plan_type': 'generator', 'plan_name': 'rel_scan', 'detectors': ['scaler1'], 'motors': ['zaxis_h'], 'num_points': 8, 'num_intervals': 7, 'plan_args': {'detectors': ["ScalerCH(prefix='gp:scaler1', name='scaler1', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan03', 'channels.chan03.s', 'channels.chan04', 'channels.

In [10]:
# What are the data streams in this run?
stream_names = run.metadata["summary"]["stream_names"]
print(f"streams: {stream_names=}")

streams: stream_names=['baseline', 'primary']


In [11]:
# Get the data from the data stream named primary (the canonical main data).
if "primary" in stream_names:
    stream_data = run["primary"]
    print(f"{stream_data['data']=}")
    # What is the metadata for this stream?
    print(f"{stream_data.metadata=}")

stream_data['data']=<DatasetClient {'time', 'timebase', 'I0', 'scint', 'diode', ...} ~8 entries>
stream_data.metadata=DictView({'descriptors': [{'run_start': 'a4c77cc5-dd49-4fc2-90e6-e3cd92667037', 'time': 1621455297.2849786, 'data_keys': {'timebase': {'source': 'PV:gp:scaler1.S1', 'dtype': 'number', 'shape': [], 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'precision': 0, 'object_name': 'scaler1'}, 'I0': {'source': 'PV:gp:scaler1.S2', 'dtype': 'number', 'shape': [], 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'precision': 0, 'object_name': 'scaler1'}, 'scint': {'source': 'PV:gp:scaler1.S3', 'dtype': 'number', 'shape': [], 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'precision': 0, 'object_name': 'scaler1'}, 'diode': {'source': 'PV:gp:scaler1.S4', 'dtype': 'number', 'shape': [], 'units': '', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'precision': 0, 'object_name': 'scaler1'}, 'scaler1_time': {'source': 'PV:gp:scaler1.T', 'd

----

Use functions from local `utils.py` file to search by date range and metadata keys and then make a nice table.

In [12]:
from utils import *

In [13]:
runs = get_tiled_runs(client["class_2021_03"], since="2021-03-17 00:30")
print(run_summary_table(runs))

== ======= ===== ======== ======= ======= ========================== ======================================
#  uid7    scan# plan     #points exit    started                    streams                               
== ======= ===== ======== ======= ======= ========================== ======================================
1  d6e1bad 25    rel_scan 23      success 2021-03-17 00:30:04.160113 baseline, primary, temperature_monitor
2  e43969f 26    rel_scan 23      success 2021-03-17 00:30:21.215641 baseline, primary, temperature_monitor
3  fb747f8 27    rel_scan 23      success 2021-03-17 00:30:27.630615 baseline, primary, temperature_monitor
4  e8c36fe 28    rel_scan 23      success 2021-03-17 00:30:33.852055 baseline, primary, temperature_monitor
5  8e2b3b5 29    count    1       success 2021-03-17 00:31:23.124599 baseline, primary                     
6  aab735d 30    count    10      success 2021-03-17 00:31:23.529676 baseline, primary                     
7  0ac9f00 31    count    10

In [14]:
runs = get_tiled_runs(client["class_2021_03"], until="2021-05-19 15:15")
print(run_summary_table(runs))

== ======= ===== ======== ======= ======= ========================== ======================================
#  uid7    scan# plan     #points exit    started                    streams                               
== ======= ===== ======== ======= ======= ========================== ======================================
1  83cae16 1     count    20      success 2021-03-15 00:52:29.556111 baseline, primary                     
2  8e78fb1 2     count    20      success 2021-03-15 11:44:21.813033 baseline, primary                     
3  933d541 3     count    20      success 2021-03-15 11:46:21.149419 baseline, primary                     
4  0ebbc98 4     count    20      success 2021-03-15 11:49:42.879512 baseline, primary                     
5  b90ceb8 5     count    20      success 2021-03-15 11:54:56.214300 baseline, primary                     
6  a501ae6 6     scan     5       success 2021-03-16 16:42:38.833505 baseline, primary                     
7  1db467b 7     scan     5 

In [15]:
runs = get_tiled_runs(client["class_2021_03"], since="2021-03-17 00:30", until="2021-05-19 15:15")
print(run_summary_table(runs))

== ======= ===== ======== ======= ======= ========================== ======================================
#  uid7    scan# plan     #points exit    started                    streams                               
== ======= ===== ======== ======= ======= ========================== ======================================
1  d6e1bad 25    rel_scan 23      success 2021-03-17 00:30:04.160113 baseline, primary, temperature_monitor
2  e43969f 26    rel_scan 23      success 2021-03-17 00:30:21.215641 baseline, primary, temperature_monitor
3  fb747f8 27    rel_scan 23      success 2021-03-17 00:30:27.630615 baseline, primary, temperature_monitor
4  e8c36fe 28    rel_scan 23      success 2021-03-17 00:30:33.852055 baseline, primary, temperature_monitor
5  8e2b3b5 29    count    1       success 2021-03-17 00:31:23.124599 baseline, primary                     
6  aab735d 30    count    10      success 2021-03-17 00:31:23.529676 baseline, primary                     
7  0ac9f00 31    count    10

In [16]:
runs = get_tiled_runs(
    client["class_2021_03"], 
    since="2021-03-17 00:30", 
    until="2021-05-19 15:15", 
    plan_name="rel_scan"
)
print(run_summary_table(runs))

= ======= ===== ======== ======= ======= ========================== ======================================
# uid7    scan# plan     #points exit    started                    streams                               
= ======= ===== ======== ======= ======= ========================== ======================================
1 d6e1bad 25    rel_scan 23      success 2021-03-17 00:30:04.160113 baseline, primary, temperature_monitor
2 e43969f 26    rel_scan 23      success 2021-03-17 00:30:21.215641 baseline, primary, temperature_monitor
3 fb747f8 27    rel_scan 23      success 2021-03-17 00:30:27.630615 baseline, primary, temperature_monitor
4 e8c36fe 28    rel_scan 23      success 2021-03-17 00:30:33.852055 baseline, primary, temperature_monitor
5 b023724 187   rel_scan 5       success 2021-05-19 15:13:47.622341 baseline, primary                     
6 a4c77cc 188   rel_scan 8       success 2021-05-19 15:14:56.577983 baseline, primary                     
= ======= ===== ======== ======= ====

Compare this last search with the results above.  Both times, 6 entries were found.

In [17]:
# One last search, for a specific `scan_id`:
runs = get_tiled_runs(client["class_2021_03"], scan_id=188)
print(run_summary_table(runs))

= ======= ===== ======== ======= ======= ========================== =================
# uid7    scan# plan     #points exit    started                    streams          
= ======= ===== ======== ======= ======= ========================== =================
1 a4c77cc 188   rel_scan 8       success 2021-05-19 15:14:56.577983 baseline, primary
= ======= ===== ======== ======= ======= ========================== =================

